# Depuracion datos Inventario Forestal Nacional

El presente notebook realiza la exploración inicial y depuración de datos del Inventario Forestal Nacional.

Se requieren los siguientes tablas enformato csv:

1. `detritos`: Contiene los mediciones realizadas en detritos de madera.

2. `ampie`: Mediciones de árboles muertos en pie.
    
3. `vegetacion`: Mediciones de árboles vivos.

4. `generalInfo`: Informacion general de cada conglomerado

5. `coordenadas`: Coordenadas de parcelas

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

# MySQL root password
password = u""

# Asignar nombres de archivos a variables
detritos = "../data/IFN/detritos.csv"
ampie =  "../data/IFN/dasometricos/amp.csv"
vegetacion = "../data/IFN/dasometricos/vegetacion.csv"
generalInfo = "../data/IFN/informacion_general/informacion_general.csv"
coordenadas = "../data/IFN/informacion_general/coordenadas.csv"

# Leer archivos como Pandas dataframes
det = pd.read_csv(detritos, encoding = 'utf8') 
amp = pd.read_csv(ampie, encoding = 'utf8')
veg = pd.read_csv(vegetacion, encoding = 'utf8')
info = pd.read_csv(generalInfo, encoding = 'utf8')
coord = pd.read_csv(coordenadas, encoding = 'utf8')

# Detritos

In [2]:
# Campos de mediciones de detritos

CONS = u"CONS" # Indice de medicion (int64)
PLOT = u"PLOT" # Indice conglomerado (int64)
TRAN = u"TRAN" # Transecto de detritos (str: 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'). Hay registros sin transectos!!!
SECC = u"SECC" # Seccion del transecto donde se registro la pieza (str: 'II', 'III', 'I'). Hay registros sin seccion!!!!
PIEZA = u"PIEZA" # Numero consecutivo de la pieza en el transecto (float64). Deberia ser int64.
TIPO = u"TIPO" # Tipo de detrito (str: 'DFM', 'DGM')
DIST = u"DIST" # Distancia del detrito en cada seccion (float64 0.0-10.0)
AZIMUT = u"AZIMUT" # Orientacion de la pieza respecto al transecto ?????????? (float64?, 0-360)
D1 = u"D1" # Primer diametro de la pieza en cm (float64)
D2 = u"D2" # Segundo diametro de la pieza en cm (float64)
INCL = u"INCL" # Inclinacion de la pieza (float64). De acuerdo al manual deberia estar en el rango [-90, 90] pero en la hoja de calculo esta sesgada a valores positivos y hay valores [90, 180].
PI_cm = u"PI_cm" # Penetracion del penetrometro en cm (float64). Verificar valores maximos.
PI_golpes = u"PI_golpes" # Golpes ejecutados con el penetrometro (float64). Por que es un numero real????
PESO_RODAJA = u"PESO_RODAJA" # Peso de la rodaja en gr (float64)
ESP1 = u"ESP1" # Primer espesor de la rodaja en cm (float64)
ESP2 = u"ESP2" # Segundo espesor de la rodaja en cm (float64)
ESP3 = u"ESP3" # Tercer espesor de la rodaja en cm (float64)
ESP4 = u"ESP4" # Cuarto espesor de la rodaja en cm (float64)
PESO_MUESTRA = u"PESO_MUESTRA" # Peso fresco de la muestra en gr (float64)
VOL = u"VOL" # Volumen de la muestra en ml (float64). Este campo no esta incluido en los formatos del INF!!!!
PESO_SECO = u"PESO_SECO" # Peso fresco de la muestra en gr (float64)
DENS = u"DENS" # Densidad de madera de la muestra en gr/ml (float64)

In [3]:
# Convertir seccion muestreo de detritos a int64 para ahorrar espacio en memoria
try:
    det[SECC].replace(to_replace = [u'I',u'II',u'III'], value = [1,2,3], inplace = True)
    # Si no existieran valores faltantes el reemplazo produciria una serie de int64, 
    # de lo contrario la columna resultante es float64
except:
    pass

for fi in [CONS, PLOT, SECC]:
    if det[fi].dtype != np.int64:
        print "Campo {0} tiene tipo inapropiado ({1} en vez de int64).".format(fi, det[fi].dtype)
        if len(det[fi][det[fi].isna()]) > 1:
            print "Valores nulos son considerados np.float64:"
            print det[[fi, PLOT]][det[fi].isna()].join(info[[u'PLOT', u'SOCIO']].set_index(PLOT), 
                                                       on=PLOT, rsuffix=u'_info')
        else:
            print "Valores np.float64 a revisar:"
            print det[fi][det[fi].map(lambda x: x % 1.0 != 0)].dropna()
            
        
for fi in [AZIMUT, PIEZA, DIST, D1, D2, INCL, PI_cm, PESO_RODAJA, ESP1, ESP2, ESP3, ESP4, 
           PESO_MUESTRA, VOL, PESO_SECO, DENS, PI_golpes]:
    if det[fi].dtype != np.float64:
        print "Campo {0} tiene tipo inapropiado ({1}en vez de float64).".format(fi, det[fi].dtype)

for fi in [TRAN, TIPO]:
    non_unicode = det[fi].dropna()[~det[fi].dropna().apply(type).eq(unicode)]
    if len(non_unicode):
        print "Campo {0} tiene tipo inapropiado ({1} en vez de unicode).".format(fi, non_strings.dtype)


Campo SECC tiene tipo inapropiado (float64 en vez de int64).
Valores nulos son considerados np.float64:
     SECC    PLOT SOCIO
897   NaN  100441  IAvH
935   NaN  124431  IAvH


In [4]:
# Indices no debe contener duplicado
if len(det[det[CONS].duplicated()]):
    print "Tabla {0} contiene indices duplicados.".format(detritos)
        
if len(det[det[TRAN].isna()]):
    print "Piezas de detritos no tienen transecto:"
    print det[[PLOT, CONS, TRAN]][det[TRAN].isna()].join(info[[u'PLOT', u'SOCIO']].set_index(PLOT), 
                                                         on=PLOT, rsuffix=u'_info')
# Piezas sin datos de transecto son eliminadas
det.drop(det[det[TRAN].isna()].index, inplace=True)
        
for tr in det[TRAN].dropna().unique():
    if tr not in [u'A', u'B', u'C', u'D', u'E', u'F', u'G', u'H']:
        print "`{0}` no es un valor valido de transecto de detrito".format(tr)
        
for tr in det[SECC].dropna().unique():
    if tr not in [1,2,3]:
        print "`{0}` no es un valor valido de transecto de detrito".format(tr)

for tr in det[TIPO].dropna().unique():
    if tr not in [u'DFM', u'DGM']:
        print "`{0}` no es un valor valido de tipo de detrito".format(tr)
        
if len(det[(det[TIPO] == u'DFM') & (((det[D1] >= 20) & (det[D2].isna())) | (det[D1] + det[D2] >= 40))]):
    print "Tipo de detrito probablemente mal asignado"
    print det[[TIPO, D1, D2, PLOT]][(det[TIPO] == u'DFM') & (((det[D1] >= 20) & (det[D2].isna()))
            | (det[D1] + det[D2] >= 40))].join(info[[u'PLOT', u'SOCIO']].set_index(PLOT), 
            on=PLOT, rsuffix=u'_info')
    det.loc[(det[TIPO] == u'DFM') & (((det[D1] >= 20) & (det[D2].isna())) | (det[D1] + det[D2] >= 40)
        ), TIPO] = u'DGM'

if len(det[(det[TIPO] == u'DGM') & (((det[D1] < 20) & (det[D2].isna())) | (det[D1] + det[D2] < 40))]):
    print "Tipo de detrito probablemente mal asignado"
    print det[[TIPO, D1, D2, PLOT]][(det[TIPO] == u'DGM') & (((det[D1] < 20) & (det[D2].isna())) 
            | (det[D1] + det[D2] < 40))].join(info[[u'PLOT', u'SOCIO']].set_index(PLOT), 
            on=PLOT, rsuffix=u'_info')
    det.loc[(det[TIPO] == u'DGM') & (((det[D1] < 20) & (det[D2].isna())) | (det[D1] + det[D2] < 40)),
            TIPO] = u'DFM'

if det[DIST].min() < 0:
    print "Rango distancia tiene valores negativos."
if det[DIST].max() > 10:
    print "Rango distancia sobrepasa el valor permitido (10)."
    print det[[DIST, PLOT]][det[DIST] > 10].join(info[[u'PLOT', u'SOCIO']].set_index(PLOT), on=PLOT, 
            rsuffix=u'_info')
    
    
##############################################
# Al parecer las medidas de distancia de DFM menores a 9 realmente fueron realizadas con el 
# punto de referencia adecuado. Teoricamente la zona de la seccion a ser considerada para
# DFM fue modificada a lo largo de la ejecucion del proyecto.
#
# Correcciones de distancias mayores a 10 
# Al parecer las medidas mayores a 10 m y menores a 30 m 
# fueron realizadas sin reestablecer el punto de 
# referencia al inicio de las secciones 2 y 3.
##############################################

det.loc[(det[DIST] > 10) & (det[DIST] <= 20), SECC] = 2
det.loc[(det[DIST] > 10) & (det[DIST] <= 20), DIST] = det[(det[DIST] > 10) & (det[DIST] <= 20)
    ][DIST] - 10

det.loc[(det[DIST] > 20) & (det[DIST] <= 30), SECC] = 3
det.loc[(det[DIST] > 20) & (det[DIST] <= 30), DIST] = det[(det[DIST] > 20) & (det[DIST] <= 30)
    ][DIST] - 20

##############################################
# Se supone que las distancias mayores a 30 fueron 
# erroneamente multiplicadas por 10
##############################################
det.loc[det[DIST] > 30, DIST] = det[det[DIST] > 30][DIST] / 10

if len(det[(det[TIPO] == u'DFM') & (det[DIST] > 1) & (det[DIST] < 9)]):
    print "Detritos finos contienen valores no permitidos de distancia:"
    print det[[TIPO, DIST, PLOT]][(det[TIPO] == u'DFM') & (det[DIST] > 1) & (det[DIST] < 9)
            ].join(info[[u'PLOT', u'SOCIO']].set_index(PLOT), on=PLOT, rsuffix=u'_info')

if det[AZIMUT].min() < 0:
    print "Rango Azimut tiene valores negativos."
if det[AZIMUT].max() > 360:
    print "Rango Azimut sobrepasa el valor permitido (360)"

#####################################################
# Valores dudosos de diametro
# Valores de diametro 0.0 deben ser nulos
det[D2].replace(to_replace = 0.0, value = np.nan, inplace = True)

# Detritos con diametro menor a 2 cm son eliminados
if len(det[(det[D1] + det[D2]) < 4]):
    print "Rango de diametro tiene valores inferiores a 2 cm."
    print det[[D1, D2, PLOT]][(det[D1] + det[D2]) < 4].join(info[[u'PLOT', u'SOCIO']
            ].set_index(PLOT), on=PLOT, rsuffix=u'_info')
det.drop(det[(det[D1] + det[D2]) < 4].index, inplace=True)
    
#
# ‚¿Como tratar los valores de inclinación? ¿Están simplemente desfazados 90 grados?
#   
if det[INCL].min() < -90:
    print "Rango inclinacion tiene valores menores al valor permitido (-90)."
    print det[INCL][det[INCL] < -90]
if det[INCL].max() > 90:
    print "Rango inclinacion sobrepasa el valor permitido (90)"
    print det[INCL][det[INCL] > 90]
    
if det[PI_cm].min() < 0:
    print "Hay valores negativos de entrada del penetrometro."
if det[PI_cm].max() > 20:
    print "Valores maximos del entrada del penetrometro mayores al valor sugerido en el manual:"
    #print det[[PI_cm, D1, D2]][det[PI_cm] > 20]
    print det[[PI_cm, PLOT]][det[PI_cm] > 20].join(info[[u'PLOT', u'SOCIO']].set_index(PLOT), 
            on=PLOT, rsuffix=u'_info')
    # Valores mayores a 20 cm son ajustados a 20 cm
    det.loc[det[PI_cm] > 20 , PI_cm] = 20

if len(det[PI_cm][(det[PI_cm] > det[D1]) | (det[PI_cm] > det[D2])]):
    print "Valores de entrada del penetrómetro son mayores al diametro registrado:"
    print det[[PI_cm, D1, D2, PLOT]][(det[PI_cm] > det[D1]) | (det[PI_cm] > det[D2])].join(
            info[[u'PLOT', u'SOCIO']].set_index(PLOT), on=PLOT, rsuffix=u'_info')

if det[PI_golpes].min() < 0:
    print "Valores negativos de golpes al penetrometro."
if det[PI_golpes].max() > 25:
    print "Valores maximos de golpes del penetrometro son dudosos:"
    print det[[PI_golpes, PLOT]][det[PI_golpes] > 20].join(info[[u'PLOT', u'SOCIO']].set_index(PLOT), 
            on=PLOT, rsuffix=u'_info')
    
# Sets de valores de espesor de pieza son dudosos si la desviacion estandar es mayor a 0.3 
# la media del set
if len(det[(det[[ESP1, ESP2, ESP3, ESP4]].std(1) / det[[ESP1, ESP2, ESP3, ESP4]].mean(1)) > 0.3]):
    print "Algunos conjuntos de espesor de pieza tienen una variacion muy alta:"
    print det[[ESP1, ESP2, ESP3, ESP4, PLOT]][(det[[ESP1, ESP2, ESP3, ESP4]].std(1) / det[[ESP1, 
                ESP2, ESP3, ESP4]].mean(1)) > 0.3].join(info[[u'PLOT', u'SOCIO']].set_index(PLOT), 
                on=PLOT, rsuffix=u'_info')
    
if det[(det[PESO_MUESTRA] < det[PESO_SECO]) | (det[PESO_MUESTRA].isna() & (det[PESO_RODAJA] < 
        det[PESO_SECO]))].size:
    print "Peso fresco es menor al peso seco:"
    print det[[PESO_MUESTRA, PESO_RODAJA, PESO_SECO, PLOT]][(det[PESO_MUESTRA] < det[PESO_SECO]) 
            | (det[PESO_MUESTRA].isna() & (det[PESO_RODAJA] < det[PESO_SECO]))].join(info[[u'PLOT', 
            u'SOCIO']].set_index(PLOT), on=PLOT, rsuffix=u'_info')

if det[(det[PESO_MUESTRA] > det[PESO_RODAJA])].size:
    print "Peso del fragmento muestreado es mayor al de la rodaja:"
    print det[[TIPO, PESO_MUESTRA, PESO_RODAJA, PESO_SECO, PLOT]][(det[PESO_MUESTRA] > 
            det[PESO_RODAJA])].join(info[[u'PLOT', u'SOCIO']].set_index(PLOT), on=PLOT, 
            rsuffix=u'_info')
    
if det[det[PESO_MUESTRA].isna() & det[PESO_RODAJA].isna()].size:
    print "Falta peso fresco de detrito:"
    print det[[TIPO, PESO_MUESTRA, PESO_RODAJA, PESO_SECO, PLOT]][det[PESO_MUESTRA].isna() & 
            det[PESO_RODAJA].isna()].join(info[[u'PLOT', u'SOCIO']].set_index(PLOT), on=PLOT, 
            rsuffix=u'_info')

Piezas de detritos no tienen transecto:
       PLOT  CONS TRAN SOCIO
897  100441   898  NaN  IAvH
935  124431   936  NaN  IAvH
Tipo de detrito probablemente mal asignado
    TIPO    D1    D2   PLOT   SOCIO
151  DFM  21.0  21.5  19031  Sinchi
174  DFM  23.5   NaN  24541  Sinchi
819  DFM  41.0   NaN  71442    IIAP
Tipo de detrito probablemente mal asignado
    TIPO    D1    D2    PLOT   SOCIO
77   DGM   8.3   8.1   14711  Sinchi
84   DGM   6.1   NaN   14711  Sinchi
469  DGM  27.9   0.0  148434    IIAP
470  DGM  26.5   0.0  148434    IIAP
471  DGM  33.0   0.0  148434    IIAP
473  DGM  24.0   0.0  148434    IIAP
486  DGM   1.6   2.0  149511    IAvH
496  DGM  20.5   8.1  153821    IAvH
497  DGM  20.0  15.9  153821    IAvH
543  DGM  20.4  19.5  173323    IAvH
546  DGM  27.0   0.0  174721    IIAP
547  DGM  27.0   0.0  176913    IIAP
549  DGM  37.0   0.0  176913    IIAP
550  DGM  38.3   0.0  176913    IIAP
611  DGM  29.3   5.0    2123  Sinchi
877  DGM  18.0   NaN   94524  Sinchi
975  DGM   3.3

# Vegetacion

In [5]:
CONS = u"CONS" # Indice de medicion comun con arboles muertos en pie (int)
PLOT = u"PLOT" # Indice conglomerado (int)
SPF = u"SPF" # Indice subparcela (int 1-5)
IND = u"IND" # Indice de individuo en el conglomerado (int)
TAMANO = u"TAMANO" # Tamaño del individuo (str: 'L', 'F', o 'FG')
AZIMUT = u"AZIMUT" # Orientacion del individuo desde el centro de la subparcela (int, 0-360)
DIST = u"DIST" # Distancia en m del individuo al centro de la parcela (float, 0-15.74)
DAP1 = u"DAP1" # Primer diámetro estimado del tallo en cm (float)
DAP2 = u"DAP2" # Segundo diámetro estimado del tallo en cm (float)
DAPA = u"DAPA" # Diametro promedio del tallo en cm (float)
ALTF = u"ALTF" # Altura fuste en m (float)
ALTT = u"ALTT" # Altura total en m (float)
FAMILIA = u"FAMILIA" # Familia taxonomica (str)
GENERO = u"GENERO" # Genero taxonomico (str)
EPITETO = u"EPITETO" # Epiteto taxonomico (str)
AUTOR = u"AUTOR" # Autor taxonomico (str)
ESPECIE = u"ESPECIE" # Binomio taxonomico (str)
DENS = u"DENS" # Densidad de la madera en gr/ml (float)
FUENTE_DENSIDAD = u"FUENTE_DENSIDAD" # Referencia bibliografica de la densidad de la madera (str)

In [6]:
for fi in [CONS, PLOT, SPF, IND]:
    if veg[fi].dtype != np.int64:
        print "\nCampo {0} tiene tipo inapropiado ({1} en vez de int64).".format(fi, veg[fi].dtype)
        if len(veg[fi][det[fi].isna()]) > 1:
            print "Valores nulos son considerados np.float64:"
            print veg[[fi, PLOT]][veg[fi].isna()].join(info[['PLOT', 'SOCIO']].set_index(PLOT), 
                                                       on=PLOT, rsuffix='_info')
        else:
            print "Valores np.float64 a revisar:"
            print veg[fi][veg[fi].map(lambda x: x % 1.0 != 0)].dropna()
            
        
for fi in [AZIMUT, DIST, DAP1, DAP2, DAPA, ALTT, ALTF, DENS]:
    if veg[fi].dtype != np.float64:
        print "\nCampo {0} tiene tipo inapropiado ({1}en vez de float64).".format(fi, veg[fi].dtype)

for fi in [FAMILIA, GENERO, EPITETO, AUTOR, ESPECIE, FUENTE_DENSIDAD, TAMANO]:
    non_strings = veg[fi].dropna()[~veg[fi].dropna().apply(type).eq(unicode)]
    if len(non_strings):
        print "\nCampo {0} tiene tipo inapropiado ({1} en vez de unicode).".format(fi, non_strings.dtype)

In [7]:
if len(veg[veg[CONS].duplicated()]):
    print "\nTabla {0} contiene indices duplicados.".format(vegetacion)

for spf in veg[SPF].unique():
    if spf not in range(1,6):
        print "\nValor no valido de parcela: {0}".format(spf)
        
for tam in veg[TAMANO].unique():
    if tam not in [u'B', u'L', u'F', u'FG']:
        print "\nValor no valido de tamaño de individuo: {0}".format(tam)
        
if veg[AZIMUT].min() < 0:
    print "\nAzimut contiene valores no aceptados."
if veg[AZIMUT].max() > 360:
    print "\nAzimut contiene valores no aceptados."
    
# Verificar asignacion de clases diametricas
if len(veg[(veg[TAMANO] != u'B') & ((veg[DAPA] < 2.5) | ((veg[DAP1] + veg[DAP2]) < 5.0))]):
    print "\nBrinzales están asignados a categoria erronea:"
    print veg[[PLOT, TAMANO, DAPA, DAP1, DAP2]][(veg[TAMANO] != u'B') & ((veg[DAPA] < 2.5) 
            | ((veg[DAP1] + veg[DAP2]) < 5.0))].join(info[[u'PLOT', u'SOCIO']].set_index(PLOT),
            on=PLOT, rsuffix=u'_info')
    veg.loc[(veg[TAMANO] != u'B') & ((veg[DAPA] < 2.5) | ((veg[DAP1] + veg[DAP2]) < 5.0)), 
        TAMANO] = u'B'
    
if len(veg[(veg[TAMANO] != u'L') & (((veg[DAPA] < 10) & (veg[DAPA] >= 2.5)) | (((veg[DAP1] + 
        veg[DAP2]) < 20) & ((veg[DAP1] + veg[DAP2]) >= 5.0)))]):
    print "\nLatizales están asignados a categoria erronea:"
    print veg[[PLOT, TAMANO, DAPA, DAP1, DAP2]][(veg[TAMANO] != u'L') & (((veg[DAPA] < 10) & 
            (veg[DAPA] >= 2.5)) | (((veg[DAP1] + veg[DAP2]) < 20) & ((veg[DAP1] + 
            veg[DAP2]) >= 5.0)))].join(info[[u'PLOT', u'SOCIO']].set_index(PLOT), on=PLOT, 
            rsuffix=u'_info')
    veg.loc[(veg[TAMANO] != u'L') & (((veg[DAPA] < 10) & (veg[DAPA] >= 2.5)) | (((veg[DAP1] 
            + veg[DAP2]) < 20) & ((veg[DAP1] + veg[DAP2]) >= 5.0))), TAMANO] = u'L'
    
if len(veg[(veg[TAMANO] != u'F') & (((veg[DAPA] < 30) & (veg[DAPA] >= 10)) | (((veg[DAP1] + 
        veg[DAP2]) < 60) & ((veg[DAP1] + veg[DAP2]) >= 20)))]):
    print "\nFustales están asignados a categoria erronea:"
    print veg[[PLOT, TAMANO, DAPA, DAP1, DAP2]][(veg[TAMANO] != u'F') & (((veg[DAPA] < 30) & 
            (veg[DAPA] >= 10)) | (((veg[DAP1] + veg[DAP2]) < 60) & ((veg[DAP1] + 
            veg[DAP2]) >= 20)))].join(info[[u'PLOT', u'SOCIO']].set_index(PLOT), on=PLOT, 
            rsuffix=u'_info')
    veg.loc[(veg[TAMANO] != u'F') & (((veg[DAPA] < 30) & (veg[DAPA] >= 10)) | (((veg[DAP1]
        + veg[DAP2]) < 60) & ((veg[DAP1] + veg[DAP2]) >= 20))) , TAMANO] = u'F'
    
if len(veg[(veg[TAMANO] != u'FG') & ((veg[DAPA] >= 30) | ((veg[DAP1] + veg[DAP2]) >= 60))]):
    print "\nFustales grandes están asignados a categoria erronea:"
    print veg[[PLOT, TAMANO, DAPA, DAP1, DAP2]][(veg[TAMANO] != u'FG') & ((veg[DAPA] >= 30) 
            | ((veg[DAP1] + veg[DAP2]) >= 60))].join(info[[u'PLOT', u'SOCIO']].set_index(PLOT),
            on=PLOT, rsuffix=u'_info')

# Verificar si las distancias corresponden a las categorias de edad.
# Latizales y Fustales afuera de su area de medición son eliminados
if len(veg[(veg[DIST] > 3) & (veg[TAMANO] == u'L')]):
    print "\nLatizales registrados afuera del area aceptada:"
    print veg[[TAMANO, DIST, DAPA, PLOT]][(veg[DIST] > 3) & (veg[TAMANO] == u'L')].join(info[[
            u'PLOT', u'SOCIO']].set_index(PLOT), on=PLOT, rsuffix='_info')
    veg.drop(veg[(veg[DIST] > 3) & (veg[TAMANO] == u'L')].index, inplace=True)
    
if len(veg[(veg[DIST] > 7) & (veg[TAMANO] == u'F')]):
    print "\nFustales registrados afuera del area aceptada:"
    print veg[[TAMANO, DIST, DAPA, PLOT]][(veg[DIST] > 7) & (veg[TAMANO] == u'F')].join(info[[
            u'PLOT', u'SOCIO']].set_index(PLOT), on=PLOT, rsuffix=u'_info')
    veg.drop(veg[(veg[DIST] > 7) & (veg[TAMANO] == u'F')].index, inplace=True)
    
if len(veg[veg[DIST] > 15]):
    print "\nIndividuos registrados afuera del area de la subparcela:"
    print veg[[TAMANO, DIST, DAPA, PLOT]][veg[DIST] > 15].join(info[[u'PLOT', u'SOCIO']]
            .set_index(PLOT), on=PLOT, rsuffix=u'_info')

# Verificar estimacion DAP promedio
if veg[((veg[DAP1] + veg[DAP2]) - (veg[DAPA] * 2)) > 0.1].size:
    print "\nErrores en la estimación del DAP promedio?:"
    print veg[[DAPA, DAP1, DAP2, PLOT]][((veg[DAP1] + veg[DAP2]) - (veg[DAPA] * 2)) > 
            0.01].join(info[[u'PLOT', u'SOCIO']].set_index(PLOT), on=PLOT, rsuffix=u'_info')

# Altura total siempre debe ser mayor a la altura del fuste
if veg[veg[ALTF] > veg[ALTT]].size:
    print "\nIndividuos con altura del fuste mayor a la altura total:"
    print veg[[ALTF, ALTT, PLOT]][veg[ALTF] > veg[ALTT]].join(info[[u'PLOT', u'SOCIO']]
            .set_index(PLOT), on = PLOT, rsuffix = u'_info')


Brinzales están asignados a categoria erronea:
         PLOT TAMANO  DAPA  DAP1  DAP2   SOCIO
585      7921      L  2.49  2.50  2.47  Sinchi
673      7921      L  2.45  2.52  2.37  Sinchi
733      9932      L  2.30  2.30  2.30  Sinchi
821      9932      L  2.29  2.08  2.50  Sinchi
822      9932      L  2.38  2.05  2.70  Sinchi
825      9932      L  2.33  2.05  2.60  Sinchi
1024    14711      L  2.45  2.50  2.40  Sinchi
1852    35132      L  2.30  2.30  2.29  Sinchi
1904    35132      L  2.47  2.44  2.50  Sinchi
2282    39534      L  2.41  2.47  2.34  Sinchi
3306    50833      L  2.06  2.06   NaN  Sinchi
3310    50833      L  2.07  2.07   NaN  Sinchi
3867    56824      L  2.46  2.46   NaN    IAvH
8060   149343      L  2.00  2.00   NaN    IAvH
10201  185244      L  2.45  2.50  2.40    IAvH

Latizales están asignados a categoria erronea:
       PLOT TAMANO  DAPA  DAP1  DAP2 SOCIO
3832  56144      F   9.6   9.6   NaN  IAvH

Fustales están asignados a categoria erronea:
       PLOT TAMANO 

# Informacion general

In [8]:
CONS = u'CONS' # Indice informacion (int64)
PLOT = u'PLOT' # Indice conglomerado (int64)
DEPARTAMENTO = u'DEPARTAMENTO' # Departamento (str)
REGION = u'REGION' # Region biogeografica (str: 'Amazonia', 'Andes', 'Pacifico', 'Orinoquia', 'Caribe')
FECHA_CAMPO = u'FECHA_CAMPO' # Año de toma de datos (int64)
SOCIO = u'SOCIO' # Institucion que ejecuta el levantamiento de datos (str: 'Sinchi', 'IAvH', 'IIAP')
BOT_TOT = u'BOT_TOT' # ???????????????? (str: 'Si', 'No'). Deberia ser boolean.
CARB = u'CARB' # Estimacion de carbono ????? (str: 'Si', 'No'). Deberia ser boolean.
SPFC = u'SPF-C' # ??????? (int64 0-5)
FERT = u'FERT' # Estimacion fertilidad ????????????? (str: 'Si', 'No'). Deberia ser boolean.
DETR = u'DETR' # Toma de detritos ????????? (str: 'Si', 'No'). Deberia ser boolean.

In [9]:
# Verificar el tipo de dato an cada campo de la tabla informacion general
for fi in [CONS, PLOT, FECHA_CAMPO, SPFC]:
    if info[fi].dtype != np.int64:
        print "Campo {0} tiene tipo inapropiado ({1} en vez de int64).".format(fi, veg[fi].dtype)
        if len(info[fi][det[fi].isna()]) > 1:
            print "Valores nulos son considerados np.float64:"
            print info[[fi, PLOT, SOCIO]]
        else:
            print "Valores np.float64 a revisar:"
            print info[fi][info[fi].map(lambda x: x % 1.0 != 0)].dropna()
            
for fi in [DEPARTAMENTO, REGION, SOCIO]:
    non_strings = info[fi].dropna()[~info[fi].dropna().apply(type).eq(unicode)]
    if len(non_strings):
        print "Campo {0} tiene tipo inapropiado ({1} en vez de unicode).".format(fi, non_strings.dtype)

###########################################
# Se asume que los campos BOT_TOT, fertilidad y detritos son boolean en ves de texto
###########################################
try:
    info[BOT_TOT].replace(to_replace = [u'Si', u'No'], value = [True, False], inplace = True)
    info[FERT].replace(to_replace = [u'Si', u'No'], value = [True, False], inplace = True)
    info[DETR].replace(to_replace = [u'Si', u'No'], value = [True, False], inplace = True)
    info[CARB].replace(to_replace = [u'Si', u'No'], value = [True, False], inplace = True)
except TypeError, ErrorMessage:
    if ErrorMessage.args[0] == "Cannot compare types 'ndarray(dtype=bool)' and 'unicode'":
        pass
except:
    raise
    
for fi in [BOT_TOT, DETR, FERT, CARB]:
    if info[fi].dtype != np.bool:
        print "Campo {0} tiene tipo inapropiado ({1} en vez de np.bool).".format(fi, veg[fi].dtype)

In [10]:
# Verificar rango de datos
if len(info[info[CONS].duplicated()]):
    print "Tabla {0} contiene indices duplicados.".format(generalInfo)

for spf in info[SPFC].unique():
    if spf not in range(1,6):
        print "\nValor no valido de parcela: {0}".format(spf)
        print info[[PLOT, SPFC, SOCIO]][info[SPFC] == spf]
        info.drop(info[info[SPFC] == spf].index, inplace=True)

if len(info[info[DETR].isna() | info[CARB].isna() | info[BOT_TOT].isna() | info[FERT].isna()]):
    print "\nDatos faltantes en columna Detritos, Carbono, Fertilidad o BOT_TOT:"
    print info[[DETR, CARB, BOT_TOT, FERT, PLOT, SOCIO]][info[DETR].isna() | info[CARB].isna() | 
            info[BOT_TOT].isna() | info[FERT].isna()]


Valor no valido de parcela: 0
       PLOT  SPF-C   SOCIO
22    38824      0    IAvH
40    52014      0    IAvH
41    52321      0    IAvH
46    56941      0    IAvH
47    57023      0    IAvH
55    72022      0    IAvH
61    86034      0    IAvH
67    88112      0    IAvH
75    96911      0    IAvH
80   100824      0    IAvH
83   103712      0  Sinchi
84   105221      0    IAvH
87   109013      0    IAvH
89   109521      0    IAvH
90   109842      0    IAvH
95   113442      0    IAvH
98   115314      0    IAvH
99   115514      0    IAvH
108  124431      0    IAvH
110  131144      0    IAvH
113  139332      0    IIAP
117  143421      0    IAvH
119  144821      0    IAvH
120  146211      0    IAvH
121  146634      0    IAvH
122  147923      0    IAvH
124  149032      0    IAvH
127  149714      0    IAvH
128  150032      0    IAvH
129  151632      0    IAvH
133  153324      0    IAvH
137  156932      0    IAvH
141  158621      0    IAvH
145  164233      0    IAvH
147  164741      0    IA

# Árboles muertos en pie

In [11]:
CONS = u'CONS' # Indice de medicion comun con arboles en pie (int64)
PLOT = u'PLOT' # Indice conglomerado (int64)
SPF = u'SPF' # Indice subparcela (int64)
TAMANO = u'TAMANO' # Tamaño del individuo (str: 'L', 'F', o 'FG')
IND = u'IND' # Indice de individuo en el conglomerado (int64)
COND = u'COND' # Condicion del individuo (str, 'MP', 'TO', 'VP', 'MC', 'M'). Valores TO, MC y M no estan consignados en el manual del INF. Que hacen individuos vivos (VP) en esta tabla?????
AZIMUT = u'AZIMUT' # Orientacion del individuo desde el centro de la subparcela (int, 0-360)
DIST = u'DIST' # Distancia en m del individuo al centro de la parcela (float, 0-15.74)
DAP_EQUIPO = u'DAP_EQUIPO' # Equipo empleado en la medicion de DAP (str: 'CD', 'FO', 'CA', 'CM'). Cuales son CM y CD? No estan especificados en el manual del INF.
DAP1 = u'DAP1' # Primer diámetro estimado del tallo en cm (float64)
DAP2 = u'DAP2' # Segundo diámetro estimado del tallo en cm (float64)
DAPA = u'DAPA' # Diametro promedio del tallo en cm (float64)
POM = u'POM' # Punto de observacion de la medida en m (float64). Hay medidas en cm.
ALT_EQUIPO = u'ALT_EQUIPO' # Equipo usado en la medicion de la altura (str: 'HI', 'VT', 'CL', 'CM', 'VX', 'FL', 'CD'). Valores 'CM', 'VX', 'FL' y 'CD' no esta especificados en el manual del INF.???????????????????
ALTF = u'ALTF' # Altura fuste en m (float)
ALTT = u'ALTT' # Altura total en m (float)
FORMA_FUSTE = u'FORMA_FUSTE' # (str: 'CIL', 'RT','IRR','FA','HI','Q'). Clases de valores estan repetidos por insercion de espacios o uso de minusculas. Valores 'HI' y 'Q' no estan consignados en el manual del INF. ??????????
DANO = u'DANO' # Daño registrado (str: 'Q', 'DB', 'SD', 'DM', 'IRR', 'EB'). Valor 'IRR' no esta consignado en el manual del INF.?????
PI_cm = u'Pi_cm' # Penetracion del penetrometro en cm (float64).
PI_golpes = u'Pi_golpes' # Golpes ejecutados con el penetrometro (float64). Por que es un numero real????

In [12]:
for fi in [CONS, PLOT, SPF, IND, AZIMUT]:
    if amp[fi].dtype != np.int64:
        print "\nCampo {0} tiene tipo inapropiado ({1} en vez de int64).".format(fi, amp[fi].dtype)
        if len(amp[fi][amp[fi].isna()]) > 1:
            print "Los siguentes valores nulos son considerados np.float64 por Pandas:"
            print amp[[fi, PLOT]][amp[fi].isna()]
        else:
            print "Valores np.float64 a revisar:"
            print amp[fi][amp[fi].map(lambda x: x % 1.0 != 0)].dropna()
            
for fi in [DIST, DAP1, DAP2, DAPA, POM, ALTF, ALTT, PI_cm, PI_golpes]:
    if amp[fi].dtype != np.float64:
        print "\nCampo {0} tiene tipo inapropiado ({1} en vez de float64).".format(fi, amp[fi].dtype)
        
        
for fi in [TAMANO, COND, DAP_EQUIPO, ALT_EQUIPO, FORMA_FUSTE, DANO]:
    non_strings = amp[fi].dropna()[~amp[fi].dropna().apply(type).eq(unicode)]
    if len(non_strings):
        print "\nCampo {0} tiene tipo inapropiado ({1} en vez de unicode).".format(fi, non_strings.dtype)




Campo AZIMUT tiene tipo inapropiado (float64 en vez de int64).
Valores np.float64 a revisar:
293    50.5
556     1.6
576     3.3
Name: AZIMUT, dtype: float64


In [13]:
if len(amp[amp[CONS].duplicated()]):
    print "Tabla {0} contiene indices duplicados.".format(ampie)

vym = pd.concat( [veg[CONS],amp[CONS]]) # Indices de vivos y muertos en pie
if len(vym[vym.duplicated()]):
    print "Existen indices duplicados en las tablas {0} y {1}.".format(veg, ampie)

for spf in amp[SPF].unique():
    if spf not in range(1,6):
        print "\nValor no valido de parcela: {0}".format(spf)
        print amp[[PLOT, SPF]][amp[SPF] == spf]
        amp.drop(amp[amp[SPF] == spf].index, inplace=True)

for tam in amp[TAMANO].unique():
    if tam not in [u'L', u'F', u'FG']:
        print "Valor no valido de tamaño de individuo: {0}".format(tam)
        
if amp[AZIMUT].min() < 0:
    print "Azimut contiene valores no aceptados."
if amp[AZIMUT].max() > 360:
    print "Azimut contiene valores no aceptados."
    
# Verificar asignacion de clases diametricas
if len(amp[(amp[TAMANO] != u'B') & ((amp[DAPA] < 2.5) | ((amp[DAP1] + amp[DAP2]) < 5.0))]):
    print "Brinzales están asignados a categoria erronea:"
    print amp[[PLOT, TAMANO, DAPA, DAP1, DAP2]][(amp[TAMANO] != u'B') & ((amp[DAPA] < 2.5) 
            | ((amp[DAP1] + amp[DAP2]) < 5.0))].join(info[[u'PLOT', u'SOCIO']].set_index(PLOT),
            on=PLOT, rsuffix=u'_info')

if len(amp[(amp[TAMANO] != u'L') & (((amp[DAPA] < 10) & (amp[DAPA] >= 2.5)) | (((amp[DAP1] + 
        amp[DAP2]) < 20) & ((amp[DAP1] + amp[DAP2]) >= 5.0)))]):
    print "Latizales están asignados a categoria erronea:"
    print amp[[PLOT, TAMANO, DAPA, DAP1, DAP2]][(amp[TAMANO] != 'L') & (((amp[DAPA] < 10) & 
            (amp[DAPA] >= 2.5)) | (((amp[DAP1] + amp[DAP2]) < 20) & ((amp[DAP1] + 
            amp[DAP2]) >= 5.0)))].join(info[[u'PLOT', u'SOCIO']].set_index(PLOT), on=PLOT, 
            rsuffix=u'_info')
    
if len(amp[(amp[TAMANO] != u'F') & (((amp[DAPA] < 30) & (amp[DAPA] >= 10)) | (((amp[DAP1] + 
        amp[DAP2]) < 60) & ((amp[DAP1] + amp[DAP2]) >= 20)))]):
    print "Fustales están asignados a categoria erronea:"
    print amp[[PLOT, TAMANO, DAPA, DAP1, DAP2]][(amp[TAMANO] != 'F') & (((amp[DAPA] < 30) & 
            (amp[DAPA] >= 10)) | (((amp[DAP1] + amp[DAP2]) < 60) & ((amp[DAP1] + 
            amp[DAP2]) >= 20)))].join(info[[u'PLOT', u'SOCIO']].set_index(PLOT), on=PLOT, 
            rsuffix=u'_info')
    amp.loc[(amp[TAMANO] != u'F') & (((amp[DAPA] < 30) & (amp[DAPA] >= 10)) | (((amp[DAP1] + 
            amp[DAP2]) < 60) & ((amp[DAP1] + amp[DAP2]) >= 20))), TAMANO] = u'F'
    
if len(amp[(amp[TAMANO] != u'FG') & ((amp[DAPA] >= 30) | ((amp[DAP1] + amp[DAP2]) >= 60))]):
    print "Fustales grandes están asignados a categoria erronea:"
    print amp[[PLOT, TAMANO, DAPA, DAP1, DAP2]][(amp[TAMANO] != 'FG') & ((amp[DAPA] >= 30) 
            | ((amp[DAP1] + amp[DAP2]) >= 60))].join(info[['PLOT', 'SOCIO']].set_index(PLOT),
            on=PLOT, rsuffix='_info')
    amp.loc[(amp[TAMANO] != u'FG') & ((amp[DAPA] >= 30) | ((amp[DAP1] + amp[DAP2]) >= 60)),
        TAMANO] = u'FG'

# Verificar si las distancias corresponden a las categorias de edad.
if len(amp[(amp[DIST] > 3) & (amp[TAMANO] == u'L')]):
    print "Latizales registrados afuera del area aceptada:"
    print amp[[TAMANO, DIST, DAPA, PLOT]][(amp[DIST] > 3) & (amp[TAMANO] == u'L')].join(info[[
            u'PLOT', u'SOCIO']].set_index(PLOT), on=PLOT, rsuffix=u'_info')
    
if len(amp[(amp[DIST] > 7) & (amp[TAMANO] == u'F')]):
    print "Fustales registrados afuera del area aceptada:"
    print amp[[TAMANO, DIST, DAPA, PLOT]][(amp[DIST] > 7) & (amp[TAMANO] == u'F')].join(info[[
            u'PLOT', u'SOCIO']].set_index(PLOT), on=PLOT, rsuffix=u'_info')
    amp.drop(amp[(amp[DIST] > 7) & (amp[TAMANO] == u'F')].index, inplace=True)
    
if len(amp[amp[DIST] > 15]):
    print "Individuos registrados afuera del area de la subparcela:"
    print amp[[TAMANO, DIST, DAPA, PLOT]][amp[DIST] > 15].join(info[[u'PLOT', u'SOCIO']]
            .set_index(PLOT), on=PLOT, rsuffix=u'_info')

# Altura total siempre debe ser mayor a la altura del fuste
if amp[amp[ALTF] > amp[ALTT]].size:
    print "Individuos con altura del fuste mayor a la altura total:"
    print amp[[ALTF, ALTT, PLOT]][amp[ALTF] > amp[ALTT]].join(info[[u'PLOT', u'SOCIO']]
            .set_index(PLOT), on=PLOT, rsuffix=u'_info')
    
# Punto de observacion de diametro
if len(amp[(amp[POM] > amp[ALTT]) | (amp[POM] > 10.0)]):
    print "\nValores de Punto de observacion de la medida mayores a la altura o mayores a 10 m."
    print amp[[ALTT, ALTF, POM, FORMA_FUSTE]][(amp[POM] > amp[ALTT]) | (amp[POM] > 10.0)]
    # Se asume que todos los valores mayores a 10 m o a la altura total fueron erroneamente
    # multiplicados por 10. Queda pendiente decidir que hacer con los valores mayores a 2 m
    # de tallo cilindrico
    amp.loc[((amp[POM] > amp[ALTT]) | (amp[POM] > 10.0)), POM] = amp[POM][(amp[POM] > amp[ALTT])] / 10.0

    
# Informacion Penetrometro
if amp[PI_cm].min() < 0:
    print "Hay valores negativos de entrada del penetrometro."
if amp[PI_cm].max() > 20:
    print "Valores maximos del entrada del penetrometro mayores al valor sugerido en el manual:"
    #print det[[PI_cm, D1, D2]][det[PI_cm] > 20]
    print amp[[PI_cm, PLOT]][amp[PI_cm] > 20].join(info[['PLOT', 'SOCIO']].set_index(PLOT), 
            on=PLOT, rsuffix='_info')

if len(amp[PI_cm][(amp[PI_cm] > amp[DAP1]) | (amp[PI_cm] > amp[DAP2])]):
    print "Valores de entrada del penetrómetro son mayores al diametro registrado:"
    print amp[[PI_cm, DAP1, DAP2, PLOT]][(amp[PI_cm] > amp[DAP1]) | (amp[PI_cm] > amp[DAP2])].join(
            info[[u'PLOT', u'SOCIO']].set_index(PLOT), on=PLOT, rsuffix=u'_info')
    # Se asume que la medida del penetrometro equivale al diametro
    amp.loc[(amp[PI_cm] > amp[DAP1]), PI_cm] = amp[DAP1][(amp[PI_cm] > amp[DAP1])]

if amp[PI_golpes].min() < 0:
    print "Valores negativos de golpes al penetrometro."
if amp[PI_golpes].max() > 25:
    print "Valores maximos de golpes del penetrometro son dudosos:"
    #print amp[PI_golpes][det[PI_golpes] > 20]
    print amp[[PI_golpes, PLOT]][amp[PI_golpes] > 20].join(info[[u'PLOT', u'SOCIO']].set_index(PLOT), 
            on=PLOT, rsuffix=u'_info')


Fustales están asignados a categoria erronea:
       PLOT TAMANO  DAPA  DAP1  DAP2   SOCIO
94    27143     FG  15.7  15.7   NaN  Sinchi
323   68521     FG  24.0  24.0   NaN  Sinchi
586  149511      L  10.0  10.0   NaN    IAvH
754  192833     FG  28.5  36.9  20.1    IAvH
Fustales grandes están asignados a categoria erronea:
       PLOT TAMANO  DAPA  DAP1  DAP2   SOCIO
203   39534      F  32.2  32.2   NaN  Sinchi
273   49214      F  31.9  31.9   NaN  Sinchi
276   50734      F  45.4  45.4   NaN  Sinchi
277   50734      F  39.8  39.8   NaN  Sinchi
279   50734      F  39.5  39.5   NaN  Sinchi
618  153821      F  52.0  52.0   NaN    IAvH
622  153821      F  46.0  46.0   NaN    IAvH
Fustales registrados afuera del area aceptada:
    TAMANO   DIST   DAPA    PLOT   SOCIO
270      F  11.00  12.00   47744    IAvH
287      F   9.97  20.00   52321     NaN
289      F   8.20  15.00   52321     NaN
293      F  15.00  11.80   56144    IAvH
295      F   9.30  27.20   56824    IAvH
296      F  10.00  28.

# Coordenadas

In [14]:
CONS = u'CONS' # Indice coordenada (int64)
PLOT = u'PLOT' # Indice conglomerado (int64)
SPF = u'SPF' # Indice subparcela dentro del conglomerado (int64 1-5)
SUBPLOT = u'SUBPLOT' # Concatenacion PLOT "_" SPF (str)
LATITUD = u'LATITUD' # Latitud en formato decimal (float64)
LONGITUD = u'LONGITUD' # Longitud en formato decimal (float64)
REGION = u'REGION' # Region biogeografica (str: 'Amazonia', 'Andes', 'Pacifico', 'Orinoquia',
                  # 'Caribe')
ZV = u'ZV' # Zona de vida???????? (int64, 3, 4, 5, 6, 7, 13, 14, 15, 19, 20, 21, 27)
ZONA_VIDA = u'ZONA_VIDA' # Zona de vida (str: "Bosque húmedo tropical",  
    # "Bosque muy húmedo tropical",  "Bosque húmedo montano bajo",  "Bosque pluvial premontano",  
    # "Bosque muy húmedo premontano",  "Bosque húmedo premontano",  "Bosque muy húmedo montano",  
    # "Bosque seco tropical",  "Bosque muy húmedo montano bajo",  "Bosque seco montano bajo",  
    # "Bosque muy seco tropical",  "Monte espinoso subtropical")
EQ = u'EQ' # Ecuacion alometrica???? (int64: 1, 2, 4, 5, 6)
E_CHAVE = u'E_CHAVE' # Coeficiente de la ecuacion de Chave (float64)

In [15]:
for fi in [CONS, PLOT, SPF, ZV, EQ]:
    if coord[fi].dtype != np.int64:
        print "\nCampo {0} tiene tipo inapropiado ({1} en vez de int64).".format(fi, coord[fi].dtype)
        if len(coord[fi][coord[fi].isna()]) > 1:
            print "Los siguentes valores nulos son considerados np.float64 por Pandas:"
            print coord[[fi, PLOT]][amp[fi].isna()]
        else:
            print "Valores np.float64 a revisar:"
            print coord[fi][coord[fi].map(lambda x: x % 1.0 != 0)].dropna()
            
            
for fi in [LATITUD, LONGITUD, E_CHAVE]:
    if coord[fi].dtype != np.float64:
        print "\nCampo {0} tiene tipo inapropiado ({1} en vez de float64).".format(fi, coord[fi].dtype)
        
        
for fi in [REGION, ZONA_VIDA]:
    non_strings = coord[fi].dropna()[~coord[fi].dropna().apply(type).eq(unicode)]
    if len(non_strings):
        print "\nCampo {0} tiene tipo inapropiado ({1} en vez de unicode).".format(fi, non_strings.dtype)

In [16]:
# Verificar que los indices no están duplicados
if len(coord[coord[CONS].duplicated()]):
    print "\nTabla {0} contiene indices duplicados.".format(coordenadas)

# Verificar rangos de coordenadas 
lat_range = (-5, 15)
lon_range = (-80, -65)
if coord[LATITUD].min() < lat_range[0] or coord[LATITUD].max() > lat_range[1]:
    print "\nLatitud fuerra de rango aceptado:"
if coord[LONGITUD].min() < lon_range[0] or coord[LONGITUD].max() > lon_range[1]:
    print "\nLongitud fuerra de rango aceptado"

# Verificar region geografica    
for re in coord[REGION].unique():
    if re not in [u'Amazonia', u'Andes', u'Pacifico', u'Orinoquia', u'Caribe']:
        print "\nRegion biogeografica no aceptada: {0}".format(re)

# Verificar que todas las parcelas están georeferenciadas
integ = info[[PLOT,SPFC]].merge(coord[[PLOT, SPF, LATITUD, LONGITUD]], on=[PLOT], how='left')
if len(integ[integ[LATITUD].isna() | integ[LONGITUD].isna()]):
    print "\nAlgunas parcelas no tienen coordenadas geograficas:"
    print integ[integ[LATITUD].isna() | integ[LONGITUD].isna()]

# Inclusion de datos via SQLAlchemy

In [17]:
import sqlalchemy as sqlalc
engine = sqlalc.create_engine('mysql+mysqldb://root:' + password + 
                              '@localhost/IFN?charset=utf8&use_unicode=1',
                              encoding='utf-8')

In [20]:
# Tabla Coordenadas

coord[[PLOT, SPF, LATITUD, LONGITUD, ZV, ZONA_VIDA, EQ, E_CHAVE]].rename(columns = {PLOT: u'Plot', SPF: u'SPF',
    LATITUD: u'Latitud', LONGITUD: u'Longitud', ZV: u'ZV', ZONA_VIDA: u'ZonaVida', EQ: u'Eq', E_CHAVE: u'ChaveE'}
    ).to_sql('Coordenadas', engine, if_exists = 'append', index = False)

In [22]:
# Tabla Conglomerados

info[[PLOT, DEPARTAMENTO, REGION, FECHA_CAMPO, SOCIO, SPFC]].rename(columns = {PLOT: u'PlotID', DEPARTAMENTO: 
    u'Departamento', REGION: u'Region', FECHA_CAMPO: u'Fecha', SOCIO: u'Socio', SPF: u'SFPC'}).to_sql('Conglomerados', 
    engine, if_exists = 'append', index = False)

OperationalError: (_mysql_exceptions.OperationalError) (1054, "Unknown column 'SPF-C' in 'field list'") [SQL: u'INSERT INTO `Conglomerados` (`PlotID`, `Departamento`, `Region`, `Fecha`, `Socio`, `SPF-C`) VALUES (%s, %s, %s, %s, %s, %s)'] [parameters: ((1324, u'Amazonas', u'Amazonia', 2016, u'Sinchi', 5), (1632, u'Amazonas', u'Amazonia', 2016, u'Sinchi', 5), (2123, u'Amazonas', u'Amazonia', 2016, u'Sinchi', 5), (2343, u'Amazonas', u'Amazonia', 2016, u'Sinchi', 5), (3641, u'Amazonas', u'Amazonia', 2016, u'Sinchi', 5), (5334, u'Amazonas', u'Amazonia', 2016, u'Sinchi', 5), (7921, u'Amazonas', u'Amazonia', 2015, u'Sinchi', 5), (9932, u'Amazonas', u'Amazonia', 2015, u'Sinchi', 5)  ... displaying 10 of 139 total bound parameter sets ...  (196532, u'La Guajira', u'Caribe', 2015, u'IAvH', 5), (197433, u'La Guajira', u'Caribe', 2016, u'IAvH', 5))]

In [ ]:
# Tabla Taxonomia

tax = {}

for row in veg[[FAMILIA, GENERO, EPITETO, AUTOR]].itertuples():
    tax[(row[1], row[2], row[3])] = row[4]
    
taxtemp = {FAMILIA:[], GENERO:[], EPITETO:[], AUTOR:[]}

for (fam, gen, epi) in tax:
    taxtemp[FAMILIA].append(fam)
    taxtemp[GENERO].append(gen)
    taxtemp[EPITETO].append(epi)
    taxtemp[AUTOR].append(tax[(fam, gen, epi)])

tax = None
taxdf = pd.DataFrame.from_dict(taxtemp)
taxtemp = None

if taxdf.index[0] == 0:
    taxdf.index += 1

taxdf.rename(columns = {FAMILIA: u'Familia', GENERO: u'Genero', AUTOR: u'Autor', EPITETO: u'Epiteto'}
    ).to_sql('Taxonomia', engine, if_exists='append', index_label=u'TaxonID')

In [ ]:
# Tabla Determinaciones

if u'Taxon' not in taxdf.columns:
    taxdf[u'Taxon'] = taxdf.index

if 'Taxon' not in veg:
    veg = veg.merge(taxdf[[FAMILIA, GENERO, EPITETO, u'Taxon']], on=[FAMILIA, GENERO, EPITETO], 
            how='left', suffixes = ['_l', '_r'])

if veg.index[0] == 0:
    veg.index += 1

veg[u'DetID'] = veg.index
veg[[u'Taxon', u'DetID']].to_sql('Determinaciones', engine, if_exists='append', index=False)

In [ ]:
amp.columns

In [ ]:
# Tabla Individuos

veg[[u'DetID', PLOT, SPF, AZIMUT, DIST]].rename(columns = {u'DetID': u'Dets', 
    PLOT: u'Plot', SPF: u'Subparcela', AZIMUT: u'Azimut', DIST: u'Distancia'}
    ).to_sql('Individuos', engine, if_exists = 'append', index = False)

amp[[PLOT, SPF, AZIMUT, DIST]].rename(columns = {PLOT: u'Plot', SPF: u'Subparcela', 
    AZIMUT: u'Azimut', DIST: u'Distancia'}).to_sql('Individuos', engine, if_exists = 
    'append', index = False)
